# Step 4-2. LangChain ReAct Agent + OpenSearch MCP
LangChain의 `create_react_agent`와 MCP(Model Context Protocol)를 사용하여 OpenSearch를 도구로 활용하는 AI 에이전트를 만듭니다.

## 핵심 개념

### ReAct Agent (Reasoning + Acting)
- LLM이 스스로 **"생각(Reasoning)"**하고, 필요한 **"행동(Acting)"**을 수행하는 패턴
- 질문을 받으면 → 어떤 도구를 사용할지 판단 → 도구 실행 → 결과를 보고 추가 행동 또는 최종 답변
- 기존 RAG(Step 4-0, 4-1)와 달리, **에이전트가 스스로 도구 사용 여부와 방법을 결정**

### MCP (Model Context Protocol)
- AI 모델이 외부 도구/데이터에 접근하기 위한 **표준 프로토콜** (by Anthropic)
- **MCP 서버**: 도구를 제공하는 서버 (이 실습에서는 OpenSearch 검색 도구)
- **MCP 클라이언트**: 도구를 사용하는 클라이언트 (LangChain 에이전트)

### Step 4-0/4-1 (RAG) vs Step 4-2 (Agent) 비교

| | RAG (Step 4-0, 4-1) | Agent + MCP (이 실습) |
|---|---|---|
| **검색 시점** | 항상 검색 후 LLM에 전달 | LLM이 필요할 때만 검색 |
| **도구 선택** | 코드에서 하드코딩 | LLM이 자율적으로 판단 |
| **유연성** | 고정된 파이프라인 | 질문에 따라 동적으로 행동 |

### 아키텍처
```
사용자 질문 → ReAct Agent (Claude LLM)
                  ↓ 스스로 판단
             MCP Client
                  ↓ stdio 통신
             MCP Server (opensearch_mcp_server.py)
                  ↓
             OpenSearch (벡터 검색)
                  ↓
             검색 결과 → Agent → 최종 답변
```

## 사전 조건
- Step 3-0 완료 → `bedrock-test` 인덱스 (텍스트 문서)
- Step 3-2b 완료 → `nova-image-test` 인덱스 (이미지)

In [ ]:
!pip install -q boto3==1.38.46 opensearch-py==2.8.0 langchain-mcp-adapters==0.2.1 langchain-aws==1.2.3 langgraph==1.0.8

## 1. 설정 (Configuration)

In [2]:
import os, sys, json

try:
    with open("../config.json") as f:
        _config = json.load(f)
    print("✅ config.json 로드 완료")
except FileNotFoundError:
    _config = {}
    print("⚠️ config.json을 찾을 수 없습니다.")
    print("   → Step 0 노트북을 먼저 실행해주세요.")

HOST = _config.get("OPENSEARCH_HOST")
if not HOST:
    raise ValueError("❌ config.json에 OPENSEARCH_HOST 값이 없습니다. Step 0 노트북을 먼저 실행해주세요.")
DEFAULT_REGION = _config.get("DEFAULT_REGION", "ap-northeast-2")
BEDROCK_REGION = _config.get("BEDROCK_REGION", "us-east-1")
PROFILE = _config.get("PROFILE", "skku-opensearch-session")

# LLM 설정 (에이전트의 두뇌)
# LLM_MODEL_ID = "us.anthropic.claude-opus-4-6-v1"
LLM_MODEL_ID = "us.anthropic.claude-sonnet-4-5-20250929-v1:0"

✅ config.json 로드 완료


## 2. OpenSearch MCP 서버 확인
`opensearch_mcp_server.py` 파일이 이 노트북과 같은 디렉토리에 있어야 합니다.

이 MCP 서버는 2개의 도구를 제공합니다:
- **`search_documents`**: 텍스트 문서를 벡터 검색 (`bedrock-test` 인덱스, Titan 임베딩)
- **`search_images`**: 이미지를 텍스트로 벡터 검색 (`nova-image-test` 인덱스, Nova 임베딩)

에이전트(LLM)가 사용자 질문을 보고 **어떤 도구를 호출할지 스스로 결정**합니다.

In [3]:
# MCP 서버 파일 존재 확인
mcp_server_path = os.path.join(os.getcwd(), "opensearch_mcp_server.py")
if os.path.exists(mcp_server_path):
    print(f"✅ MCP 서버 파일 확인: {mcp_server_path}")
else:
    print(f"❌ MCP 서버 파일을 찾을 수 없습니다: {mcp_server_path}")
    print("   → opensearch_mcp_server.py 파일이 이 노트북과 같은 디렉토리에 있는지 확인해주세요.")

✅ MCP 서버 파일 확인: /Users/gnidoc327/ws/side/skku-kdt-opensearch-25/example/python/step4/opensearch_mcp_server.py


## 3. ReAct 에이전트 생성
1. **ChatBedrockConverse**: Bedrock의 Claude 모델을 LangChain 호환 LLM으로 생성
2. **MultiServerMCPClient**: MCP 서버를 subprocess로 시작하고 도구를 로드
3. **create_react_agent**: LLM + 도구를 결합하여 ReAct 에이전트 생성

In [4]:
from langchain_mcp_adapters.client import MultiServerMCPClient
from langgraph.prebuilt import create_react_agent
from langchain_aws import ChatBedrockConverse

# 1. LLM 생성 (에이전트의 두뇌)
model = ChatBedrockConverse(
    model_id=LLM_MODEL_ID,
    region_name=BEDROCK_REGION,
    credentials_profile_name=PROFILE,
)
print(f"✅ LLM 생성 완료: {LLM_MODEL_ID}")

# 2. MCP 서버 시작 및 도구 로드
#    환경변수로 OpenSearch/Bedrock 접속 정보를 MCP 서버에 전달
server_env = os.environ.copy()
server_env.update({
    "OPENSEARCH_HOST": HOST,
    "DEFAULT_REGION": DEFAULT_REGION,
    "BEDROCK_REGION": BEDROCK_REGION,
    "AWS_PROFILE": PROFILE,
})

mcp_client = MultiServerMCPClient({
    "opensearch": {
        "transport": "stdio",
        "command": sys.executable,
        "args": ["opensearch_mcp_server.py"],
        "env": server_env,
    }
})

tools = await mcp_client.get_tools()

print(f"\n✅ MCP 서버에서 {len(tools)}개의 도구를 로드했습니다:")
for tool in tools:
    print(f"   - {tool.name}: {tool.description[:80]}...")

# 3. ReAct 에이전트 생성
agent = create_react_agent(model, tools)
print("\n✅ ReAct 에이전트 생성 완료")

/Users/gnidoc327/ws/side/skku-kdt-opensearch-25/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ LLM 생성 완료: us.anthropic.claude-sonnet-4-5-20250929-v1:0

✅ MCP 서버에서 2개의 도구를 로드했습니다:
   - search_documents: OpenSearch에서 질문과 관련된 기술 문서를 벡터 검색합니다.
기술 블로그, 개발 관련 질문에 답변할 때 사용하세요.
...
   - search_images: OpenSearch에서 질문과 관련된 이미지를 벡터 검색합니다.
이미지를 찾거나, 동물/사물 등 시각적 콘텐츠를 검색할 때 사용하세요.
한국어와...

✅ ReAct 에이전트 생성 완료


/var/folders/h0/gd6q0wjd04n6gnqr0s0h3ml00000gn/T/ipykernel_58829/1283197333.py:39: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  agent = create_react_agent(model, tools)


## 4. 에이전트 실행
에이전트에게 질문을 보내면, 에이전트가 스스로 판단하여 적절한 도구를 호출하고 최종 답변을 생성합니다.

**스트리밍 출력**으로 에이전트의 전체 사고 과정(ReAct Loop)을 실시간으로 확인할 수 있습니다:
1. 사용자 질문 수신
2. 어떤 도구를 사용할지 판단 → 도구 호출 (Reasoning + Acting)
3. 도구 결과 수신
4. 최종 답변을 **토큰 단위로 스트리밍** 생성

In [5]:
async def ask_agent(query):
    """에이전트에게 질문하고, 사고 과정을 스트리밍으로 출력합니다."""
    print(f"{'='*60}")
    print(f"👤 사용자: {query}")
    print(f"{'='*60}\n")

    is_answering = False

    async for event in agent.astream_events(
        {"messages": [("user", query)]},
        version="v2",
    ):
        kind = event["event"]

        if kind == "on_tool_start":
            tool_input = event["data"].get("input", {})
            # 내부 runtime 데이터 제외하고 실제 인자만 추출
            filtered = {k: v for k, v in tool_input.items() if k != "runtime"}
            try:
                input_str = json.dumps(filtered, ensure_ascii=False)
            except TypeError:
                input_str = str(filtered)
            print(f"🤖 에이전트 판단 → '{event['name']}' 도구 호출")
            print(f"   인자: {input_str}\n")

        elif kind == "on_tool_end":
            output = str(event["data"].get("output", ""))
            # ToolMessage content에서 텍스트만 추출
            if "text" in output and "[1]" in output:
                # 검색 결과 요약만 표시
                start = output.find("[1]")
                end = output.find("', 'id':")
                if start >= 0 and end >= 0:
                    preview = output[start:end]
                else:
                    preview = output[start:start+300]
                preview = preview[:300] + "..." if len(preview) > 300 else preview
            else:
                preview = output[:300] + "..." if len(output) > 300 else output
            print(f"🔧 [{event['name']}] 결과:")
            print(f"   {preview}\n")

        elif kind == "on_chat_model_stream":
            chunk = event["data"]["chunk"]
            # 최종 답변 텍스트를 스트리밍 출력
            text = ""
            if isinstance(chunk.content, str):
                text = chunk.content
            elif isinstance(chunk.content, list):
                for block in chunk.content:
                    if isinstance(block, dict) and block.get("type") == "text":
                        text += block.get("text", "")
            if text:
                if not is_answering:
                    print(f"{'='*60}")
                    print("🤖 최종 답변:")
                    print(f"{'='*60}")
                    is_answering = True
                print(text, end="", flush=True)

    if is_answering:
        print("\n")  # 스트리밍 출력 후 줄바꿈

### 테스트 1: 문서 검색이 필요한 질문
에이전트가 `search_documents` 도구를 선택하는지 확인해보세요.

In [6]:
# TODO: 질문을 바꿔보세요!
await ask_agent("Docker와 Kubernetes의 차이점에 대해 알려줘")

👤 사용자: Docker와 Kubernetes의 차이점에 대해 알려줘

🤖 에이전트 판단 → 'search_documents' 도구 호출
   인자: {"query": "Docker Kubernetes 차이점 비교"}

🔧 [search_documents] 결과:
   [1] (유사도: 1.900) Docker와 Kubernetes의 차이점이 뭔가요?\n작성자: newbie_coder\n내용: 컨테이너 기술을 공부하고 있는데, Docker와 Kubernetes의 역할이 헷갈립니다. 쉽게 설명해주실 수 있을까요?\n\n[2] (유사도: 1.529) 가상화(Virtualization)와 컨테이너(Container) 기술의 근본적인 차이\n작성자: system_pro\n내용: VMware나 VirtualBox 같은 가상화 기술과 Docker 같은 컨테이너 기술 모두 격리된 환경을 제공하는데, 하이퍼바이...

🤖 최종 답변:
Docker와 Kubernetes의 차이점에 대해 설명드리겠습니다.

## **Docker와 Kubernetes의 주요 차이점**

### **1. 기본 역할**

- **Docker**: 컨테이너화 플랫폼
  - 애플리케이션을 컨테이너로 패키징하고 실행하는 도구
  - 단일 호스트에서 컨테이너를 생성, 실행, 관리
  
- **Kubernetes (K8s)**: 컨테이너 오케스트레이션 플랫폼
  - 여러 컨테이너를 대규모로 관리하고 조율하는 시스템
  - 다수의 호스트에서 컨테이너를 자동으로 배포, 확장, 관리

### **2. 사용 목적**

| Docker | Kubernetes |
|--------|-----------|
| 컨테이너 생성 및 실행 | 컨테이너 배포 및 관리 자동화 |
| 개발 환경 구축 | 프로덕션 환경 운영 |
| 단일 서버 운영 | 클러스터 환경 운영 |

### **3. 주요 기능 비교**

**Docker:**
- 컨테이너 이미지 빌드
- 컨테이너 실행 및 중지
- 로컬 개발 환경 구성
- Docker Compo

### 테스트 2: 이미지 검색이 필요한 질문
에이전트가 `search_images` 도구를 선택하는지 확인해보세요.

In [7]:
# TODO: 질문을 바꿔보세요!
await ask_agent("강아지 이미지를 찾아줘")

👤 사용자: 강아지 이미지를 찾아줘

🤖 에이전트 판단 → 'search_images' 도구 호출
   인자: {"query": "강아지"}

🔧 [search_images] 결과:
   [1] (유사도: 1.421) 이미지 경로: ../../data/image/dog_3.jpg\n\n[2] (유사도: 1.416) 이미지 경로: ../../data/image/dog_2.jpg\n\n[3] (유사도: 1.321) 이미지 경로: ../../data/image/dog_1.png

🤖 최종 답변:
강아지 이미지를 찾았습니다! 총 3개의 이미지가 검색되었습니다:

1. **dog_3.jpg** (유사도: 1.421)
2. **dog_2.jpg** (유사도: 1.416)
3. **dog_1.png** (유사도: 1.321)

모두 강아지 관련 이미지이며, 유사도가 높은 순서로 정렬되어 있습니다. 특정 종류의 강아지나 더 구체적인 이미지를 찾으시나요?



### 테스트 3: 도구가 필요 없는 일반 질문
에이전트가 도구를 사용하지 않고 직접 답변하는지 확인해보세요.

In [8]:
# TODO: 질문을 바꿔보세요!
await ask_agent("Python에서 리스트와 튜플의 차이점은?")

👤 사용자: Python에서 리스트와 튜플의 차이점은?

🤖 에이전트 판단 → 'search_documents' 도구 호출
   인자: {"query": "Python 리스트 튜플 차이점 list tuple difference"}

🔧 [search_documents] 결과:
   [1] (유사도: 1.204) Docker와 Kubernetes의 차이점이 뭔가요?\n작성자: newbie_coder\n내용: 컨테이너 기술을 공부하고 있는데, Docker와 Kubernetes의 역할이 헷갈립니다. 쉽게 설명해주실 수 있을까요?\n\n[2] (유사도: 1.188) 웹소켓(WebSocket)과 HTTP의 차이점\n작성자: backend_ninja\n내용: 실시간 채팅 기능을 구현하려고 합니다. 일반적인 HTTP 폴링 방식 대신 웹소켓을 사용하는 이유가 무엇인가요? 양방향 통신과 연결 유지 측면에서 어떤 장점이 있는지

🤖 최종 답변:
검색 결과가 정확하지 않네요. Python 리스트와 튜플의 차이점에 대해 직접 설명해드리겠습니다.

## Python 리스트(List)와 튜플(Tuple)의 주요 차이점

### 1. **가변성(Mutability)**
- **리스트(List)**: 가변(mutable) - 생성 후 요소 추가, 삭제, 수정 가능
- **튜플(Tuple)**: 불변(immutable) - 생성 후 변경 불가능

```python
# 리스트 - 변경 가능
my_list = [1, 2, 3]
my_list[0] = 10  # OK
my_list.append(4)  # OK

# 튜플 - 변경 불가능
my_tuple = (1, 2, 3)
my_tuple[0] = 10  # TypeError 발생!
```

### 2. **표기법**
- **리스트**: 대괄호 `[]` 사용
- **튜플**: 소괄호 `()` 사용 (또는 생략 가능)

```python
my_list = [1, 2, 3]
my_tuple = (1, 2, 3)
my_tuple2 = 1, 2, 3  

## 5. 정리
MCP 서버와의 연결을 종료합니다. 노트북을 다 사용한 후 실행해주세요.

에이전트를 다시 사용하려면 **섹션 3부터 다시 실행**하면 됩니다.

In [9]:
del mcp_client, agent, tools
print("✅ 정리 완료. 에이전트를 다시 사용하려면 섹션 3부터 다시 실행하세요.")

✅ 정리 완료. 에이전트를 다시 사용하려면 섹션 3부터 다시 실행하세요.


## 실습 과제

### 과제 1: 다양한 질문으로 에이전트 행동 관찰
- 에이전트가 **어떤 도구를 선택하는지** 관찰해보세요
- 문서 검색과 이미지 검색을 **동시에 요구하는 질문**은 어떻게 처리하는지 확인해보세요
  - 예: `"벌 이미지를 찾고, 관련 기술 문서도 검색해줘"`

### 과제 2: LLM 모델 변경 후 비교
위 설정 셀에서 `LLM_MODEL_ID`를 변경하고, 에이전트를 다시 생성(섹션 3부터 재실행)해보세요:
- `us.anthropic.claude-sonnet-4-5-20250929-v1:0` (기본, 빠름)
- `us.anthropic.claude-opus-4-6-v1` (고성능, 느림)

### 과제 3: MCP 서버에 새로운 도구 추가
`opensearch_mcp_server.py`에 새로운 도구를 추가해보세요:
- 예: `list_indices()` - OpenSearch에 존재하는 인덱스 목록을 반환
- 도구를 추가한 후 노트북을 섹션 3부터 다시 실행하면 에이전트가 새 도구도 사용할 수 있습니다